In [3]:
import json, re, ast

In [9]:
FASHION_METAFILE="../data/fashion_prefab_metadata_all.json"
FURNITURE_METAFILE="../data/furniture_prefab_metadata_all.json"

with open(FASHION_METAFILE) as f:
    fash_meta = json.load(f)

with open(FURNITURE_METAFILE) as f:
    fur_meta = json.load(f)

len(fash_meta) + len(fur_meta)

345

In [58]:
name2id_fash = dict()
for id, name in enumerate(fash_meta):
    name2id_fash[name] = id

name2id_fur = dict()
for id, name in enumerate(fur_meta):
    name2id_fur[name] = id

id2name_fash = dict()
for id, name in enumerate(fash_meta):
    id2name_fash[id] = name

id2name_fur = dict()
for id, name in enumerate(fur_meta):
    id2name_fur[id] = name

print(name2id_fash['WomensCollection/Prefabs/Rearranged/leather_jacket_1'])
print(id2name_fash[116])
print(fash_meta[id2name_fash[116]]['brand'])
print(fur_meta[id2name_fur[16]]['brand'])

116
WomensCollection/Prefabs/Rearranged/leather_jacket_1
Coats & More
212 Local


In [59]:
def insert_attributes(line):
    pos = 0
    idx = line.find("<@", pos)
    while idx != -1:
        # get absolute object ID
        abs_id = line[idx+2:idx+6]
        # get object type
        meta = abs_id[0]
        abs_id = int(abs_id[1:])
        obj_type = fash_meta[id2name_fash[abs_id]]['brand'] if meta=='1' else fur_meta[id2name_fur[abs_id]]['brand']
        # append object type to line
        line = line[:idx+7] + obj_type + line[idx+7:]
        # update pos and idx
        pos = idx+4
        idx = line.find("<@", pos)
    return line

In [60]:
with open("../models/bart_coref/data_object_special/simmc2_dials_dstc10_train_predict.txt", encoding="utf-8") as f:
    for line in f.read().splitlines():
        if (len(line) > 0 and not line.isspace()):
            line_boxes = [ast.literal_eval(position.replace('(', '').replace(')', '')) for position in re.findall(r"\[\([^\)]+\)\]", line)]
            line = re.sub(r"\[\([^\)]*\)\]", "", line)
            l = insert_attributes(line)
            print("Line before insertion:", line)
            print("Line after insertion:", l)
            break

Line before insertion: User : Hi, do you have any jackets today? <SOO><NOCOREF><OBJ><59><@1116><OBJ><80><@1127><OBJ><55><@1050><OBJ><67><@1132><OBJ><68><@1227><OBJ><70><@1235><OBJ><82><@1052><OBJ><79><@1283><OBJ><69><@1225><OBJ><72><@1137><OBJ><71><@1233><OBJ><73><@1109><OBJ><61><@1013><OBJ><62><@1015><OBJ><63><@1015><OBJ><58><@1018><OBJ><74><@1283><OBJ><75><@1166><OBJ><64><@1053><OBJ><76><@1067><EOO> => Belief State : 
Line after insertion: User : Hi, do you have any jackets today? <SOO><NOCOREF><OBJ><59><@1116>Coats & More<OBJ><80><@1127>The Vegan Baker<OBJ><55><@1050>Global Voyager<OBJ><67><@1132>Pedals & Gears<OBJ><68><@1227>212 Local<OBJ><70><@1235>The Vegan Baker<OBJ><82><@1052>Pedals & Gears<OBJ><79><@1283>Uptown Gallery<OBJ><69><@1225>Uptown Gallery<OBJ><72><@1137>HairDo<OBJ><71><@1233>Downtown Consignment<OBJ><73><@1109>Cats Are Great<OBJ><61><@1013>Downtown Consignment<OBJ><62><@1015>Modern Arts<OBJ><63><@1015>Modern Arts<OBJ><58><@1018>Yogi Fit<OBJ><74><@1283>Uptown Gallery<